In [1]:
import glob
import os
from random import shuffle
from shutil import copyfile
import numpy as np
import json

In [2]:
def subfiles(folder, join = True, prefix = None, suffix = None, sort = True):
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isfile(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

def save_json(obj, file: str, indent: int = 4, sort_keys: bool = True) -> None:
    with open(file, 'w') as f:
        json.dump(obj, f, sort_keys=sort_keys, indent=indent)

In [20]:
img = glob.glob('/Users/liamchalcroft/Desktop/MRES/Data/CT-MR/train/MR/train/img*.nii.gz') + \
    glob.glob('/Users/liamchalcroft/Desktop/MRES/Data/CT-MR/train/MR/val/img*.nii.gz')
lab = [img_.replace('img', 'lab') for img_ in img]

img_test = glob.glob('/Users/liamchalcroft/Desktop/MRES/Data/CT-MR/train/MR/test/img*.nii.gz')
lab_test = [img_.replace('img', 'lab') for img_ in img_test]

In [21]:
# img = glob.glob('/home/lchalcroft/Data/ATLAS_R2.0/Training/R*/sub-r*/ses-*/anat/sub*_T1w.nii.gz')
# # img  = img[:16]
# lab = [img_.replace('T1w', 'label-L_desc-T1lesion_mask') for img_ in img]
# img = ['/'+os.path.join(*image.split('/')[:-1])+'/n4_stripped.nii.gz' for image in img]

# img_test = glob.glob('/home/lchalcroft/Data/ATLAS_R2.0/Testing/R*/sub-r*/ses-*/anat/sub*_T1w.nii.gz')
# img_test = ['/'+os.path.join(*image.split('/')[:-1])+'/n4_stripped.nii.gz' for image in img_test]
# # img_test = img[:4]

In [22]:
odir = '/Users/liamchalcroft/Desktop/MRES/nnunet-test/nnUNet_raw_data_base/nnUNet_raw_data/Task002_META'
name = 'META'

In [23]:
os.makedirs(os.path.join(odir,'imagesTr'), exist_ok=True)
os.makedirs(os.path.join(odir,'metaTr'), exist_ok=True)
os.makedirs(os.path.join(odir,'labelsTr'), exist_ok=True)
os.makedirs(os.path.join(odir,'imagesTs'), exist_ok=True)
os.makedirs(os.path.join(odir,'metaTs'), exist_ok=True)
os.makedirs(os.path.join(odir,'labelsTs'), exist_ok=True)

In [24]:
# for i, img_ in enumerate(img):
#     copyfile(img_,os.path.join(odir,'imagesTr',name+'_%04d_0000.nii.gz' % (i+1)))

for i, (img_, lab_) in enumerate(zip(img, lab)):
    copyfile(img_,os.path.join(odir,'imagesTr',name+'_%04d_0000.nii.gz' % (i+1)))
    copyfile(lab_,os.path.join(odir,'labelsTr',name+'_%04d.nii.gz' % (i+1)))
    if img_.split('/')[-3]=='MR':
        np.save(os.path.join(odir,'metaTr',name+'_%04d.npy' % (i+1)), np.array([1.,0.]))
    elif img_.split('/')[-3]=='CT':
        np.save(os.path.join(odir,'metaTr',name+'_%04d.npy' % (i+1)), np.array([0.,1.]))
    else:
        raise IndexError

for i, (img_, lab_) in enumerate(zip(img_test, lab_test)):
    copyfile(img_,os.path.join(odir,'imagesTs',name+'_%04d_0000.nii.gz' % (i+1)))
    copyfile(lab_,os.path.join(odir,'labelsTs',name+'_%04d.nii.gz' % (i+1)))
    if img_.split('/')[-3]=='MR':
        np.save(os.path.join(odir,'metaTs',name+'_%04d.npy' % (i+1)), np.array([1.,0.]))
    elif img_.split('/')[-3]=='CT':
        np.save(os.path.join(odir,'metaTs',name+'_%04d.npy' % (i+1)), np.array([0.,1.]))
    else:
        raise IndexError

In [25]:
def get_identifiers_from_splitted_files(folder: str):
    uniques = np.unique([i[:-12] for i in subfiles(folder, suffix='.nii.gz', join=False)])
    return uniques

In [26]:
train_identifiers = get_identifiers_from_splitted_files(os.path.join(odir,'imagesTr'))
test_identifiers = get_identifiers_from_splitted_files(os.path.join(odir,'imagesTs'))

json_dict = {}
json_dict['name'] = name
json_dict['description'] = 'Stroke Lesion Segmentation'
json_dict['tensorImageSize'] = "4D"
json_dict['reference'] = "BLANK"
json_dict['licence'] = 'BLANK'
json_dict['release'] = 'BLANK'
json_dict['modality'] = {'0': 'T1'}
json_dict['labels'] = {'0': 'Background', '1': 'Lesion'}

json_dict['numTraining'] = len(train_identifiers)
json_dict['numTest'] = len(test_identifiers)
json_dict['training'] = [
    {'image': os.path.join(odir,"imagesTr/%s.nii.gz" % i), 
    "label": os.path.join(odir,"labelsTr/%s.nii.gz" % i),
    "meta": os.path.join(odir,"metaTr/%s.npy" % i)} for i
    in
    train_identifiers]
json_dict['test'] = [{'image': os.path.join(odir,"imagesTs/%s.nii.gz" % i), 
    "meta": os.path.join(odir,"metaTs/%s.npy" % i)} for i in test_identifiers]

save_json(json_dict, os.path.join(odir,'dataset.json'))

In [19]:
json_dict['training'][0]

{'image': '/Users/liamchalcroft/Desktop/MRES/nnunet-test/nnUNet_raw_data_base/nnUNet_raw_data/Task002_META/imagesTr/META_0001.nii.gz',
 'label': '/Users/liamchalcroft/Desktop/MRES/nnunet-test/nnUNet_raw_data_base/nnUNet_raw_data/Task002_META/labelsTr/META_0001.nii.gz',
 'meta': '/Users/liamchalcroft/Desktop/MRES/nnunet-test/nnUNet_raw_data_base/nnUNet_raw_data/Task002_META/metaTr/META_0001.npy'}